In [4]:
!pip install pulp


   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ---------- ----------------------------- 4.2/16.4 MB 35.7 MB/s eta 0:00:01
   ---------------------------------------  16.3/16.4 MB 48.6 MB/s eta 0:00:01
   ---------------------------------------- 16.4/16.4 MB 38.2 MB/s eta 0:00:00


In [6]:

from pulp import LpProblem, LpVariable, LpMinimize, LpStatus, lpSum

#Defining Hub cities (name, current tons, capacity)
hubs = {
    "CVG": {"current_tons": 82800, "capacity": 95650},
    "AFW": {"current_tons": 38400, "capacity": 44350}
}

#Defining Focus Cities (name, airport, capacity)
focus_cities = {
    "Leipzig": {"airport": "Leipzig/Halle Airport", "capacity": 85000},
    "Hyderabad": {"airport": "Rajiv Gandhi International Airport", "capacity": 19000},
    "San Bernardino": {"airport": "San Bernardino International Airport", "capacity": 36000}
}

#Defining centers (country, city, demand)
centers = {
    "France": {"Paris": 6500},
    "Germany": {"Cologne": 640, "Hanover": 180},
    "India": {"Bangalore": 9100, "Coimbatore": 570, "Delhi": 19000, "Mumbai": 14800},
    "Italy": {"Cagliari": 90, "Catania": 185, "Milan": 800, "Rome": 1700},
    "Poland": {"Katowice": 170},
    "Spain": {"Barcelona": 2800, "Madrid": 3700},
    "United Kingdom": {"Castle Donington": 30, "London": 6700},
    "United States": {
        "Mobile": 190, "Anchorage": 175, "Fairbanks": 38, "Phoenix": 2400, "Los Angeles": 7200, "Ontario": 100,
        "Riverside": 1200, "Sacramento": 1100, "San Francisco": 1900, "Stockton": 240, "Denver": 1500, "Hartford": 540,
        "Miami": 3400, "Lakeland": 185, "Tampa": 1600, "Atlanta": 3000, "Honolulu": 500, "Kahului/Maui": 16,
        "Kona": 63, "Chicago": 5100, "Rockford": 172, "Fort Wayne": 200, "South Bend": 173, "Des Moines": 300,
        "Wichita": 290, "New Orleans": 550, "Baltimore": 1300, "Minneapolis": 1700, "Kansas City": 975, "St. Louis": 1200,
        "Omaha": 480, "Manchester": 100, "Albuquerque": 450, "New York": 11200, "Charlotte": 900, "Toledo": 290,
        "Wilmington": 150, "Portland": 1200, "Allentown": 420, "Pittsburgh": 1000, "San Juan": 1100, "Nashville": 650,
        "Austin": 975, "Dallas": 3300, "Houston": 3300, "San Antonio": 1100, "Richmond": 600, "Seattle/Tacoma": 2000,
        "Spokane": 260}}

In [8]:
#Flattening the centers
def flatten_centers(centers):
    return {city: demand for country, cities in centers.items() for city, demand in cities.items()}
flattened_centers = flatten_centers(centers)

#Default cost (high number for unconnected cities so they cannot be seen as efficient)
DEFAULT_COST = 10000

#Fill all missing costs with default value 10000
def get_cost(src, dest, cost_data, default_cost=10000):
    return cost_data.get(src, {}).get(dest, default_cost)

In [10]:
#Defining the cost between hubs, focus cities, and centers
cost = {
    "Cincinnati/NKY": {
        "Leipzig": 1.5,
        "Hyderabad": DEFAULT_COST,
        "San Bernardino": 0.5,
        "Paris": 1.6,
        "Cologne": 1.5,
        "Hanover": 1.5,
        "Bengaluru": DEFAULT_COST,
        "Coimbatore": DEFAULT_COST,
        "Delhi": DEFAULT_COST,
        "Mumbai": DEFAULT_COST,
        "Cagliari": 1.5,
        "Catania": 1.5,
        "Milan": 1.5,
        "Rome": 1.5,
        "Katowice": 1.4,
        "Barcelona": 1.5,
        "Madrid": 1.6,
        "Castle Donington": 1.4,
        "London": 1.6,
        "Mobile": 0.5,
        "Anchorage": 1.3,
        "Fairbanks": 1.4,
        "Phoenix": 0.5,
        "Los Angeles": 0.5,
        "Ontario": 0.5,
        "Riverside": 0.5,
        "Sacramento": 0.5,
        "San Francisco": 0.5,
        "Stockton": 0.5,
        "Denver": 0.5,
        "Hartford": 0.5,
        "Miami": 0.5,
        "Lakeland": 0.5,
        "Tampa": 0.5,
        "Atlanta": 0.5,
        "Honolulu": DEFAULT_COST,
        "Kahului/Maui": DEFAULT_COST,
        "Kona": DEFAULT_COST,
        "Chicago": 0.5,
        "Chicago/Rockford": 0.5,
        "Fort Wayne": 0.5,
        "South Bend": 0.5,
        "Des Moines": 0.5,
        "Wichita": 0.5,
        "New Orleans": 0.5,
        "Baltimore": 0.5,
        "Minneapolis": 0.5,
        "Kansas City": 0.5,
        "St. Louis": 0.5,
        "Omaha": 0.5,
        "Manchester": 0.5,
        "Albuquerque": 0.5,
        "New York": 0.5,
        "Charlotte": 0.5,
        "Toledo": 0.5,
        "Wilmington": 0.5,
        "Portland": 0.5,
        "Allentown": 0.5,
        "Pittsburgh": 0.5,
        "San Juan": 0.5,
        "Nashville": 0.5,
        "Austin": 0.5,
        "Dallas/Fort Worth": 0.5,
        "Houston": 0.5,
        "San Antonio": 0.5,
        "Richmond": 0.5,
        "Seattle/Tacoma": 0.5,
        "Spokane": 0.5,
    },
    "Alliance Fort Worth": {
        "Leipzig": DEFAULT_COST,
        "Hyderabad": DEFAULT_COST,
        "San Bernardino": 0.5,
        "Paris": DEFAULT_COST,
        "Cologne": DEFAULT_COST,
        "Hanover": DEFAULT_COST,
        "Bengaluru": DEFAULT_COST,
        "Coimbatore": DEFAULT_COST,
        "Delhi": DEFAULT_COST,
        "Mumbai": DEFAULT_COST,
        "Cagliari": DEFAULT_COST,
        "Catania": DEFAULT_COST,
        "Milan": DEFAULT_COST,
        "Rome": DEFAULT_COST,
        "Katowice": DEFAULT_COST,
        "Barcelona": DEFAULT_COST,
        "Madrid": DEFAULT_COST,
        "Castle Donington": DEFAULT_COST,
        "London": DEFAULT_COST,
        "Mobile": 0.5,
        "Anchorage": 1,
        "Fairbanks": 1,
        "Phoenix": 0.5,
        "Los Angeles": 0.5,
        "Ontario": 0.5,
        "Riverside": 0.5,
        "Sacramento": 0.5,
        "San Francisco": 0.5,
        "Stockton": 0.5,
        "Denver": 0.5,
        "Hartford": 1.5,
        "Miami": 0.5,
        "Lakeland": 0.5,
        "Tampa": 0.5,
        "Atlanta": 0.5,
        "Honolulu": 0.5,
        "Kahului/Maui": 0.5,
        "Kona": 0.5,
        "Chicago": 0.5,
        "Chicago/Rockford": 0.5,
        "Fort Wayne": 0.5,
        "South Bend": 0.5,
        "Des Moines": 0.5,
        "Wichita": 0.5,
        "New Orleans": 0.5,
        "Baltimore": 1.5,
        "Minneapolis": 0.5,
        "Kansas City": 0.5,
        "St. Louis": 0.5,
        "Omaha": 0.5,
        "Manchester": 1.5,
        "Albuquerque": 0.5,
        "New York": 1.6,
        "Charlotte": 0.5,
        "Toledo": 0.5,
        "Wilmington": 0.7,
        "Portland": 0.5,
        "Allentown": 1.5,
        "Pittsburgh": 0.5,
        "San Juan": 0.5,
        "Nashville": 0.5,
        "Austin": 0.25,
        "Dallas/Fort Worth": 0.5,
        "Houston": 0.25,
        "San Antonio": 0.25,
        "Richmond": 0.5,
        "Seattle/Tacoma": 0.5,
        "Spokane": 0.5,
    },
    "Leipzig": {
        "Hyderabad": 1.6,
        "San Bernardino": DEFAULT_COST,
        "Paris": 0.5,
        "Cologne": 0.5,
        "Hanover": 0.5,
        "Bengaluru": 1.5,
        "Coimbatore": 1.5,
        "Delhi": 1.5,
        "Mumbai": 1.5,
        "Cagliari": 0.5,
        "Catania": 0.5,
        "Milan": 0.5,
        "Rome": 0.5,
        "Katowice": 0.5,
        "Barcelona": 0.5,
        "Madrid": 0.5,
        "Castle Donington": DEFAULT_COST,
        "London": 0.75,
        "Mobile": DEFAULT_COST,
        "Anchorage": DEFAULT_COST,
        "Fairbanks": DEFAULT_COST,
        "Phoenix": DEFAULT_COST,
        "Los Angeles": DEFAULT_COST,
        "Ontario": DEFAULT_COST,
        "Riverside": DEFAULT_COST,
        "Sacramento": DEFAULT_COST,
        "San Francisco": DEFAULT_COST,
        "Stockton": DEFAULT_COST,
        "Denver": DEFAULT_COST,
        "Hartford": DEFAULT_COST,
        "Miami": DEFAULT_COST,
        "Lakeland": DEFAULT_COST,
        "Tampa": DEFAULT_COST,
        "Atlanta": DEFAULT_COST,
        "Honolulu": DEFAULT_COST,
        "Kahului/Maui": DEFAULT_COST,
        "Kona": DEFAULT_COST,
        "Chicago": DEFAULT_COST,
        "Chicago/Rockford": DEFAULT_COST,
        "Fort Wayne": DEFAULT_COST,
        "South Bend": DEFAULT_COST,
        "Des Moines": DEFAULT_COST,
        "Wichita": DEFAULT_COST,
        "New Orleans": DEFAULT_COST,
        "Baltimore": DEFAULT_COST,
        "Minneapolis": DEFAULT_COST,
        "Kansas City": DEFAULT_COST,
        "St. Louis": DEFAULT_COST,
        "Omaha": DEFAULT_COST,
        "Manchester": DEFAULT_COST,
        "Albuquerque": DEFAULT_COST,
        "New York": DEFAULT_COST,
        "Charlotte": DEFAULT_COST,
        "Toledo": DEFAULT_COST,
        "Wilmington": DEFAULT_COST,
        "Portland": DEFAULT_COST,
        "Allentown": DEFAULT_COST,
        "Pittsburgh": DEFAULT_COST,
        "San Juan": DEFAULT_COST,
        "Nashville": DEFAULT_COST,
        "Austin": DEFAULT_COST,
        "Dallas/Fort Worth": DEFAULT_COST,
        "Houston": DEFAULT_COST,
        "San Antonio": DEFAULT_COST,
        "Richmond": DEFAULT_COST,
        "Seattle/Tacoma": DEFAULT_COST,
        "Spokane": DEFAULT_COST,
    }
}

In [12]:
#Defining the optimization and variables
prob = LpProblem("Cargo_Optimization", LpMinimize)

x = LpVariable.dicts("shipment_from_hub_to_focus", (hubs.keys(), focus_cities.keys()), lowBound=0, cat='Continuous')
y = LpVariable.dicts("shipment_from_focus_to_center", (focus_cities.keys(), flattened_centers.keys()), lowBound=0, cat='Continuous')

#Defining the objective function to minimize cost
prob += lpSum([get_cost(h, f, cost) * x[h][f] for h in hubs for f in focus_cities]) + \
         lpSum([get_cost(f, c, cost) * y[f][c] for f in focus_cities for c in flattened_centers]), "Total_Shipping_Cost"

In [14]:
#Defining a list of constraints

#1. Hub capacity
for h in hubs:
    prob += lpSum([x[h][f] for f in focus_cities]) <= hubs[h]["capacity"], f"Hub_{h}_Capacity"

#2. Focus city capacity
for f in focus_cities:
    prob += lpSum([x[h][f] for h in hubs]) <= focus_cities[f]["capacity"], f"FocusCity_{f}_Capacity"

#3. Center demand
for city, demand in flattened_centers.items():
    prob += lpSum([y[f][city] for f in focus_cities]) == demand, f"Demand_{city}"

#4. Flow balance for focus cities
for f in focus_cities:
    prob += lpSum([x[h][f] for h in hubs]) == lpSum([y[f][city] for city in flattened_centers]), f"Flow_Balance_{f}"

#Solve the optimization and output results
prob.solve()

if LpStatus[prob.status] == "Optimal":
    for h in hubs:
        for f in focus_cities:
            if x[h][f].varValue > 0:
                print(f"Ship {x[h][f].varValue} tons from {h} to {f}")
    for f in focus_cities:
        for city in flattened_centers:
            if y[f][city].varValue > 0:
                print(f"Ship {y[f][city].varValue} tons from {f} to {city}")
else:
    print("No optimal solution found")

Ship 85000.0 tons from CVG to Leipzig
Ship 4397.0 tons from CVG to Hyderabad
Ship 8350.0 tons from AFW to Hyderabad
Ship 36000.0 tons from AFW to San Bernardino
Ship 6500.0 tons from Leipzig to Paris
Ship 640.0 tons from Leipzig to Cologne
Ship 180.0 tons from Leipzig to Hanover
Ship 9100.0 tons from Leipzig to Bangalore
Ship 570.0 tons from Leipzig to Coimbatore
Ship 19000.0 tons from Leipzig to Delhi
Ship 14800.0 tons from Leipzig to Mumbai
Ship 90.0 tons from Leipzig to Cagliari
Ship 185.0 tons from Leipzig to Catania
Ship 800.0 tons from Leipzig to Milan
Ship 1700.0 tons from Leipzig to Rome
Ship 170.0 tons from Leipzig to Katowice
Ship 2800.0 tons from Leipzig to Barcelona
Ship 3700.0 tons from Leipzig to Madrid
Ship 30.0 tons from Leipzig to Castle Donington
Ship 6700.0 tons from Leipzig to London
Ship 190.0 tons from Leipzig to Mobile
Ship 175.0 tons from Leipzig to Anchorage
Ship 1927.0 tons from Leipzig to Phoenix
Ship 7200.0 tons from Leipzig to Los Angeles
Ship 1100.0 tons f

In [16]:
#Checking constraints: hub capacity
for h in hubs:
    total_shipment_from_hub = sum(x[h][f].varValue for f in focus_cities)
    if total_shipment_from_hub <= hubs[h]["capacity"]:
        print(f"Hub {h} capacity constraint satisfied: {total_shipment_from_hub} <= {hubs[h]['capacity']}")
    else:
        print(f"Hub {h} capacity constraint violated: {total_shipment_from_hub} > {hubs[h]['capacity']}")

Hub CVG capacity constraint satisfied: 89397.0 <= 95650
Hub AFW capacity constraint satisfied: 44350.0 <= 44350


In [18]:
#Checking constraints: focus city capacity
for f in focus_cities:
    total_shipment_to_focus_city = sum(x[h][f].varValue for h in hubs)
    if total_shipment_to_focus_city <= focus_cities[f]["capacity"]:
        print(f"Focus city {f} capacity constraint satisfied: {total_shipment_to_focus_city} <= {focus_cities[f]['capacity']}")
    else:
        print(f"Focus city {f} capacity constraint violated: {total_shipment_to_focus_city} > {focus_cities[f]['capacity']}")

Focus city Leipzig capacity constraint satisfied: 85000.0 <= 85000
Focus city Hyderabad capacity constraint satisfied: 12747.0 <= 19000
Focus city San Bernardino capacity constraint satisfied: 36000.0 <= 36000


In [20]:
#Checking constraints: center demand
for city, demand in flattened_centers.items():
    total_shipment_to_center = sum(y[f][city].varValue for f in focus_cities)
    if total_shipment_to_center == demand:
        print(f"Center {city} demand constraint satisfied: {total_shipment_to_center} == {demand}")
    else:
        print(f"Center {city} demand constraint violated: {total_shipment_to_center} != {demand}")

Center Paris demand constraint satisfied: 6500.0 == 6500
Center Cologne demand constraint satisfied: 640.0 == 640
Center Hanover demand constraint satisfied: 180.0 == 180
Center Bangalore demand constraint satisfied: 9100.0 == 9100
Center Coimbatore demand constraint satisfied: 570.0 == 570
Center Delhi demand constraint satisfied: 19000.0 == 19000
Center Mumbai demand constraint satisfied: 14800.0 == 14800
Center Cagliari demand constraint satisfied: 90.0 == 90
Center Catania demand constraint satisfied: 185.0 == 185
Center Milan demand constraint satisfied: 800.0 == 800
Center Rome demand constraint satisfied: 1700.0 == 1700
Center Katowice demand constraint satisfied: 170.0 == 170
Center Barcelona demand constraint satisfied: 2800.0 == 2800
Center Madrid demand constraint satisfied: 3700.0 == 3700
Center Castle Donington demand constraint satisfied: 30.0 == 30
Center London demand constraint satisfied: 6700.0 == 6700
Center Mobile demand constraint satisfied: 190.0 == 190
Center Anc

In [22]:
#Checking constraints: flow balance for focus cities
for f in focus_cities:
    total_flow_in = sum(x[h][f].varValue for h in hubs)
    total_flow_out = sum(y[f][city].varValue for city in flattened_centers)
    if total_flow_in == total_flow_out:
        print(f"Flow balance for focus city {f} satisfied: {total_flow_in} == {total_flow_out}")
    else:
        print(f"Flow balance for focus city {f} violated: {total_flow_in} != {total_flow_out}")

Flow balance for focus city Leipzig satisfied: 85000.0 == 85000.0
Flow balance for focus city Hyderabad satisfied: 12747.0 == 12747.0
Flow balance for focus city San Bernardino satisfied: 36000.0 == 36000.0
